In [1]:
import setup
import os
import numpy as np
from decouple import config, AutoConfig
config = AutoConfig(search_path="/home/harry/chatbotDjango") 

setup.init_django()

In [2]:
from coffeshop.models import EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement
from coffeshop import services
from llama_index.core.schema import TextNode

In [3]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

In [4]:
from mistralai import Mistral

MISTRAL_API_KEY = config("MISTRAL_API_KEY")

EMEDDING_LENGTH=config("EMEDDING_LENGTH", default=1024, cast=int)

LLM_CONFIG = {
    "api_key" : MISTRAL_API_KEY,
}

EMBED_CONFIG = {
    "api_key" : MISTRAL_API_KEY,
    "model_name" : "mistral-embed"
}

llm = MistralAI(**LLM_CONFIG)
embed_model = MistralAIEmbedding(**EMBED_CONFIG)

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
models = [
    EmployeeRole,
    Employee,
    ProductType,
    Product,
    InventoryItem,
    ProductInventoryRequirement
]

table_names = [model.__name__.lower() + "s" for model in models]
vector_db_name = "vector_db"

In [7]:
from decouple import config, AutoConfig
config = AutoConfig(search_path="/home/harry/chatbotDjango") 
DATABASE_URL = config("DATABASE_URL_POOL")
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

In [8]:
from sqlalchemy import create_engine, text

engine = create_engine(DATABASE_URL, isolation_level="AUTOCOMMIT")
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1 FROM pg_database WHERE datname = :db_name"), {"db_name": vector_db_name})
    db_exists = result.scalar() == 1
    
    if not db_exists:
        connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector WITH SCHEMA public"))
        connection.execute(text(f"CREATE DATABASE {vector_db_name}"))

In [9]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

url = make_url(DATABASE_URL)
vector_stores = {}

for model, table_name in zip(models, table_names):
    vector_store = PGVectorStore.from_params(
        database=vector_db_name,
        host=url.host,
        password=url.password,
        port=url.port or 5432,
        user=url.username,
        table_name=table_name,
        embed_dim=EMEDDING_LENGTH,
    )
    vector_stores[model.__name__] = vector_store

In [10]:
from llama_index.core import VectorStoreIndex, StorageContext

indices = {}
query_engines = {}

for model, table_name in zip(models, table_names):
    instances = model.objects.all()
    
    nodes = [
        TextNode(
            text=str(instance),
            metadata={
                "model": model.__name__,
                "id": instance.id,
                "created_at": instance.created_at.isoformat() if hasattr(instance, 'created_at') else None
            }
        ) for instance in instances
    ]
    
    storage_context = StorageContext.from_defaults(
        vector_store=vector_stores[model.__name__]
    )
    indices[model.__name__] = VectorStoreIndex(
        nodes, storage_context=storage_context
    )
    query_engines[model.__name__] = indices[model.__name__].as_query_engine()

In [11]:
from llama_index.core import Document
from datetime import datetime

def should_include_in_index(obj) -> bool:
    """Filter logic for indexing"""
    if hasattr(obj, 'is_active') and not obj.is_active:
        return False
    if hasattr(obj, 'is_available') and not obj.is_available:
        return False
    return True
RELEVANT_MODELS = [Employee, ProductType, Product, InventoryItem]

docs = []
for model in RELEVANT_MODELS:
    qs = model.objects.all()
    for obj in qs:
        if not should_include_in_index(obj):
            continue
            
        docs.append(Document(
            text=obj.get_embedding_text_raw(),
            doc_id=f"{model.__name__.lower()}_{obj.id}",
            metadata={
                "model_type": model.__name__,
                "pk": obj.pk,
                "last_updated": datetime.now().isoformat(),
                **{
                    field.name: getattr(obj, field.name)
                    for field in model._meta.fields
                    if field.name in ['name', 'description', 'hire_date', 'price']
                }
            }
        ))

In [12]:
from llama_index.core import StorageContext

def handle_document_update(doc: Document):
    model_name = doc.metadata["model_type"]
    index = indices.get(model_name)
    vector_store = vector_stores.get(model_name)
    
    if not index or not vector_store:
        print(f"Skipping document for unconfigured model: {model_name}")
        return

    try:
        with StorageContext.from_defaults(vector_store=vector_store) as storage_context:
            index.delete_ref_doc(doc.doc_id)
            index.insert(doc, storage_context=storage_context)
            model = globals()[model_name]
            obj = model.objects.get(pk=doc.metadata["pk"])
            obj.last_indexed = datetime.now()
            obj.save(update_fields=['last_indexed'])
            
    except Exception as e:
        print(f"Failed processing {doc.doc_id}: {str(e)}")

from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(handle_document_update, docs)

In [13]:
employee_role_response = query_engines["EmployeeRole"].query(
    "List all employee roles and their permissions"
)
print(f"Employee Roles Response: {employee_role_response}")

product_response = query_engines["Product"].query(
    "What are the top 5 most expensive products?"
)
print(f"Products Response: {product_response}")

Employee Roles Response: Empty Response
Products Response: Empty Response
